# Saving Algerian Constitution 2020 into SurrealDB

## 1. Importing libraries

In [9]:
%pip install surrealdb==1.0.3 openai==1.55.2 python-dotenv==1.0.1

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datetime import datetime
from dotenv import load_dotenv
from surrealdb import Surreal
from openai import OpenAI
import json
import os

True

## 2. Load the data

In [26]:
json_file_path = os.path.join("json", "dz_consti_2020.json")
with open(json_file_path, "r", encoding="utf-8") as f:
    constitution_data = json.load(f)

## 3. Connect to the database

In [36]:
load_dotenv(override=True)

db = Surreal(os.getenv("SURREAL_URL"))
db.signin({"username": os.getenv("SURREAL_USER"), "password": os.getenv("SURREAL_PASS")})
db.use(os.getenv("SURREAL_NAMESPACE"), os.getenv("SURREAL_DATABASE"))

print(db.version())

surrealdb-2.2.0


## 4. Embed articles and save to database

In [ ]:
client = OpenAI() # api key loaded from .env
embed_model_name = "text-embedding-3-small"

def save_constitution():
    titles = constitution_data['titles'][1:]
    for title in titles:
        chapters = title['chapters']
        for chapter in chapters:
            articles = chapter['articles']
            for article in articles:
                # more about this on https://platform.openai.com/docs/guides/embeddings#how-to-get-embeddings
                response = client.embeddings.create(
                    input=f"{title['header']}: {chapter['header']}: {article}",
                    model=embed_model_name,
                )
                vector = response.data[0].embedding

                db.insert(
                    "articles",
                    [{
                        "text": article,
                        "embed_vector": vector,
                        "chapter": chapter['header'],
                        "title": title['header'],
                        "edition": datetime(2020, 1, 1).strftime('%Y-%m-%dT%H:%M:%SZ'),
                    }]
                )


save_constitution()

## 5. Test Search in Database

In [ ]:
# user_question = "worshiping in Algeria"
user_question = "Qu'est-ce que je peux faire dans la guerre?"

response = client.embeddings.create(
    input=user_question,
    model=embed_model_name,
)
q_vector = response.data[0].embedding

results = db.query("SELECT text, vector::similarity::cosine(embed_vector, $q_vector) AS score FROM articles ORDER BY score DESC LIMIT 10", {"q_vector": q_vector})

print(json.dumps(results, indent=2, ensure_ascii=False))

[
  {
    "score": 0.21770371660865162,
    "text": "المادة 29 : تمتنع الجزائر عن اللّجوء إلى الحرب من أجل المساس بالسّيادة المشروعة للشّعوب الأخرى وحرّيّتها. وتبذل جهدها لتسوية الخلافات الدّوليّة بالوسائل السّلميّة."
  },
  {
    "score": 0.1928271882325211,
    "text": "المادة 169 : الحقّ في الدّفاع معترف به. الحقّ في الدّفاع مضمون في القضايا الجزائيّة."
  },
  {
    "score": 0.18120915981811977,
    "text": "المادة 28 : تنتظم الطّاقة الدّفاعيّة للأمّة، ودعمها، وتطويرها، حول الجيش الوطنيّ الشّعبيّ. تتمثّل المهمّة الدّائمة للجيش الوطنيّ الشّعبيّ في المحافظة على الاستقلال الوطنيّ، والدّفاع عن السّيادة الوطنيّة. كما يضطلع بالدّفاع عن وحدة البلاد، وسلامتها التّرابيّة، وحماية مجالها البرّيّ والجوّيّ، ومختلف مناطق أملاكها البحريّة."
  },
  {
    "score": 0.1785322494551577,
    "text": "المادة 71  : الحقّ في الإضراب مُعترَف به، ويُمارَس في إطار القانون. يمكن أن يمنع القانون ممارسة هذا الحقّ، أو يجعل حدودا لممارسته في ميادين الدّفاع الوطنيّ والأمن، أو في جميع الخدمات أو الأعمال العموميّة ذا